<a href="https://colab.research.google.com/github/akashsharma-2002/NLP_text_summarization./blob/main/NLP_text_summarizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import torch
from transformers import BertModel, BertTokenizer
from nltk.tokenize import sent_tokenize
from rouge import Rouge


WRAPPING OUTPUT IN COLLAB CELL


In [42]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''

  '''))
get_ipython().events.register('pre_run_cell', set_css)

install transformers with **sentencepiece**

In [43]:

!pip install transformers[sentencepiece]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


OPENING THE TRANSCRIPT FILE DOWNLOADED AND READING IT WHICH WILL BE USED FOR SUMMARIZATION

In [63]:
file = open("/content/drive/MyDrive/Colab Notebooks/new_transcription.txt", "r")
FileContent = file.read().strip()

READING THE WHOLE CONTENT INSIDE THE TRANSCRIPT

In [64]:
FileContent

'"Yeah. Yeah, sure. It kinda does make sense, doesn\'t it, because when we get into the end of meeting we\'re kind of talking about action and design as opposed to background. Everything I have is kinda background. Mm-hmm. Uh that sounds. Sure. Okay. Sure. Yeah, cool. Why don\'t I get that? Hmm. Okay. Okay. Um alright so c is it function F_ eight? Hmm. Come on. I think it\'s working. Okay great s so let me just start this. Okay great. So um uh s move on. Uh-huh oh where\'d it all go? It\'s not good. Okay lemme just see where I can find it. This looks more like it. I think I just opened up the template. Sorry about that. Okay alright so let\'s have a look here. Okay so this was the method that um I\'ve taken. Uh basically what I wanna do here, before we get into it uh too far, is I want to show you all the background information I have that I think we need to acknowledge if we want this to be successful. And uh and then sorta g go through some of the way that I\'ve dealt with that infor

# THE REFERENCE SUMMARY OF THE TEXT IS SAVED IN REF

In [85]:
ref="""Market research shows that TV remote control has a fancy look and feel, not a functional look or or feel. Number one priority was that it be innovative without a adding unnecessary functional bits to it. The number one priority for ease of use is that it has to be, um quite user friendly while still having technology.
 Style is number one thing in the in the market of who we're selling to. Innovative design technology's also a must in that it's seen it'd be seen to be uh cutting edge, but ease of use t has to be insured throughout. And then at the end there are vibrant natural colours.
 We need to have something that unifies a lot of the different concepts, and if we think that what we are w our number one marketing motive is the look and feel. We are leaning quite a bit to the side of being low-tech, rubber buttons plastic frame, it's almost like we're reproducing the remote control that's out there. We're gonna need to put in a really good battery so people don't have to charge their r remote control every few days.
 Could we use like a jog dial, like a nice just sort of round, round, somewhere on it where you just roll it? Or a mouse then? Well why don't we do it like a mouse? Mm-hmm. Jog dials are much easier than that, the alternative is to go push the button.
 Andrew and Craig and David lead the meeting to discuss the product. Andrew says the final objective of the meeting is to reach a decision on the concepts of the product and the result hopefully appear in a second. Craig says he'd quite like a sort of snowman type shape. Andrew: 'I think we could go for like a p a fruit shape'
 Mm-hmm. I think it's yellow because like the website is yellow and there's a band at the bottom is yellow, so yellow, lemon, you know definitely food for thought there, but keep going and we'll discuss it after. Mm 'kay. Do y do you want to sit in the the line of sight of this um Yeah. Do you want something like s could sit in your hand comfortably, or do you th you'd hold onto comfortably or So something quite curvy? Mm. Okay. A snowman shape? Uh-huh. That's quite a distinctive shape.
 Or when you like or if you turn it off or something if it can speak if it could actually say the slogan it might be a bit more powerful than just having it written on it somewhere. I d I d any thoughts on that at all? I don't know if there is really, no um I would say no need for a talk-back. Or if we wanted to incorporate an L_C_D_ display, where would we put that on the inside or It's bound to increase the cost of it a lot, I would've thought. I think that would make it very complex. So so no. need for an L.C_DD_ display? I think
 The most input that's needed is basically in the user interface. The rest of the components do have an impact in terms of cost and complexity. The power is basically a factor of that. And the lower components, the power, the logic, the transmitter, and the infrared, um they affect you. The case can actually be flat or it can be curved, and then the different types of materials that you can use.
 The kinetic one I guess for me is the most interesting one because it's movement and people like to fiddle with their and it's a nice sales gimmick I think. From a marketing gimmick it it it is a technology thing. The less components we use and the simpler the components means you reduce your cost and you increase your profit. Um and the other thing is from a production point of view.
 The case is transparent so it gives it a little bit of a glow, but in the dark it sort of, it's alive. Or or there might be a light running through it like a mouse. The question is when you're rolling it, how do you want 'em to roll it like that?"""

TOTAL CHAR IN FILE

In [65]:
len(FileContent)

39727

# LOAD THE MODEL AND TOKENIZER

In [66]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [67]:
tokenizer.model_max_length

1024

In [68]:
tokenizer.max_len_single_sentence

1022

NUMBER OF SPECIAL TOKEN

In [69]:
tokenizer.num_special_tokens_to_add()

2

# CONVERTING FILE CONTENT TO SENTENCES

In [70]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


find the max tokens in the longest sentence

In [71]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

93

# CREATING CHUNCKS AND COUNTING THE NUMBER OF CHUNKS

In [72]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

10

In [73]:

[len(tokenizer.tokenize(c)) for c in chunks]

[1015, 984, 960, 1005, 1016, 994, 991, 1005, 1017, 576]

In [74]:

[len(tokenizer(c).input_ids) for c in chunks]

[1017, 986, 962, 1007, 1018, 996, 993, 1007, 1019, 578]

In [75]:

sum([len(tokenizer(c).input_ids) for c in chunks])

9583

SPECIAL TOKEN ADDED

In [76]:

len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (9565 > 1024). Running this sequence through the model will result in indexing errors


9565

In [77]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

9563

In [78]:

len(tokenizer.tokenize(FileContent))

9563

# GETTING INPUT

In [79]:

inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

# PRINTING THE OUTPUT FOR THE GIVEN INPUT

THIS IS SUMMARY OF THE TRANSCRIPT

In [82]:

for input in inputs:
  output = model.generate(**input)
  actual=(tokenizer.decode(*output, skip_special_tokens=True))
  print(actual)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 Market research shows that TV remote control has a fancy look and feel, not a functional look or or feel. Number one priority was that it be innovative without a adding unnecessary functional bits to it. The number one priority for ease of use is that it has to be, um quite user friendly while still having technology.
 Style is number one thing in the in the market of who we're selling to. Innovative design technology's also a must in that it's seen it'd be seen to be uh cutting edge, but ease of use t has to be insured throughout. And then at the end there are vibrant natural colours.
 We need to have something that unifies a lot of the different concepts, and if we think that what we are w our number one marketing motive is the look and feel. We are leaning quite a bit to the side of being low-tech, rubber buttons plastic frame, it's almost like we're reproducing the remote control that's out there. We're gonna need to put in a really good battery so people don't have to charge thei

GENERATING F1 SCORE FOR THE MODEL BUILT

In [83]:
from rouge import Rouge

In [86]:
reference_summaries = ref # List of reference summaries
generated_summaries = actual # List of generated summaries


In [87]:
rouge = Rouge()

In [88]:
scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

In [89]:
rouge_1_f1 = scores['rouge-1']['f']

# Extract ROUGE-2 F1 score
rouge_2_f1 = scores['rouge-2']['f']

# Extract ROUGE-L F1 score
rouge_l_f1 = scores['rouge-l']['f']


In [90]:
print("ROUGE-1 F1:", rouge_1_f1)
print("ROUGE-2 F1:", rouge_2_f1)
print("ROUGE-L F1:", rouge_l_f1)

ROUGE-1 F1: 0.2330623285642732
ROUGE-2 F1: 0.1461318038030476
ROUGE-L F1: 0.2330623285642732
